# Projectile Motion - Parametric Study

This notebook demonstrates parametric analysis of projectile motion using the Funz-Modelica plugin.

## Problem Description

We simulate a projectile launched with initial velocity $v_0$ at angle $\theta$ from horizontal:

**Equations of motion:**
$$\frac{dx}{dt} = v_x$$
$$\frac{dy}{dt} = v_y$$
$$\frac{dv_x}{dt} = 0$$
$$\frac{dv_y}{dt} = -g$$

Where:
- $(x, y)$ is position
- $(v_x, v_y)$ is velocity
- $g = 9.81$ m/s² is gravitational acceleration

We will study how launch angle and initial velocity affect trajectory and range.

## 1. Create the Modelica Model

We define a parametric model where angle and velocity can be varied by Funz.

In [ ]:
# Create the Modelica model file
model_content = '''model ProjectileMotion
  "Parametric projectile motion"
  
  // Parametric variables (can be varied by Funz)
  parameter Real v0 = ${velocity~20.0} "Initial velocity (m/s)";
  parameter Real angle = ${launch_angle~45.0} "Launch angle (degrees)";
  
  // Fixed parameters
  parameter Real g = 9.81 "Gravitational acceleration (m/s^2)";
  parameter Real m = 1.0 "Mass (kg)";
  
  // State variables
  Real x(start = 0.0) "Horizontal position (m)";
  Real y(start = 0.0) "Vertical position (m)";
  Real vx(start = v0 * cos(angle * 3.14159265359 / 180.0)) "Horizontal velocity (m/s)";
  Real vy(start = v0 * sin(angle * 3.14159265359 / 180.0)) "Vertical velocity (m/s)";
  
equation
  // Equations of motion
  der(x) = vx;
  der(y) = vy;
  der(vx) = 0;  // No horizontal acceleration
  der(vy) = -g; // Gravitational acceleration downward
  
end ProjectileMotion;
'''

# Write model to file
with open('ProjectileMotion.mo', 'w') as f:
    f.write(model_content)

print("✓ Modelica model created: ProjectileMotion.mo")
print(f"\nParametric variables:")
print(f"  - velocity (default=20.0 m/s)")
print(f"  - launch_angle (default=45.0°)")

## 2. Study Effect of Launch Angle

Let's first study how launch angle affects the trajectory with fixed velocity.

In [ ]:
import fz
import pandas as pd
import numpy as np

# Run parametric study varying angle
print("Running parametric study: Effect of launch angle...")
angles = [5, 15, 30, 45, 60, 75, 85]

results_angle = fz.fzr(
    "ProjectileMotion.mo",
    {"launch_angle": angles, "velocity": 20.0},
    "Modelica",
    calculators="localhost",
    results_dir="results_angle"
)

print("\nResults summary:")
display(results_angle[['launch_angle', 'velocity', 'status']])

### Visualize Trajectories for Different Angles

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 8))

# Plot trajectories
for idx, row in results_angle.iterrows():
    if row['status'] == 'done' and 'res' in row:
        angle = row['launch_angle']
        res_data = row['res']['ProjectileMotion']
        
        x_vals = list(res_data['x'].values())
        y_vals = list(res_data['y'].values())
        
        # Only plot while y >= 0 (before landing)
        valid_points = [(x, y) for x, y in zip(x_vals, y_vals) if y >= -0.1]
        if valid_points:
            x_plot, y_plot = zip(*valid_points)
            plt.plot(x_plot, y_plot, marker='o', markersize=3, linewidth=2.5,
                    label=f'θ = {angle}°', alpha=0.8)

plt.xlabel('Horizontal Distance (m)', fontsize=13)
plt.ylabel('Height (m)', fontsize=13)
plt.title('Projectile Motion - Effect of Launch Angle (v₀ = 20 m/s)', 
          fontsize=15, fontweight='bold')
plt.legend(fontsize=11, loc='upper right')
plt.grid(True, alpha=0.3)
plt.xlim(left=0)
plt.ylim(bottom=0)
plt.tight_layout()
plt.savefig('trajectories_angle.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Plot saved as 'trajectories_angle.png'")

### Calculate Range and Maximum Height

In [ ]:
# Calculate key metrics for each angle
angle_analysis = []

for idx, row in results_angle.iterrows():
    if row['status'] == 'done' and 'res' in row:
        angle = row['launch_angle']
        res_data = row['res']['ProjectileMotion']
        
        x_vals = np.array(list(res_data['x'].values()))
        y_vals = np.array(list(res_data['y'].values()))
        t_vals = np.array(list(res_data['time'].values()))
        
        # Find landing point (where y crosses 0)
        landing_idx = np.where(y_vals < 0)[0]
        if len(landing_idx) > 0:
            landing_idx = landing_idx[0] - 1
        else:
            landing_idx = len(y_vals) - 1
        
        range_distance = x_vals[landing_idx]
        flight_time = t_vals[landing_idx]
        max_height = np.max(y_vals[y_vals >= 0])
        
        angle_analysis.append({
            'angle': angle,
            'range_m': range_distance,
            'max_height_m': max_height,
            'flight_time_s': flight_time
        })

angle_df = pd.DataFrame(angle_analysis)
print("\nAngle Analysis (v₀ = 20 m/s):")
print("="*70)
display(angle_df.round(2))

# Find optimal angle for range
optimal_idx = angle_df['range_m'].idxmax()
optimal_angle = angle_df.loc[optimal_idx, 'angle']
max_range = angle_df.loc[optimal_idx, 'range_m']

print(f"\n🎯 Optimal angle for maximum range: {optimal_angle}° (range = {max_range:.2f} m)")

### Visualize Range vs Angle

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Range vs angle
ax1.plot(angle_df['angle'], angle_df['range_m'], 'o-', linewidth=2.5, 
         markersize=10, color='blue')
ax1.axvline(x=optimal_angle, color='red', linestyle='--', linewidth=2,
           label=f'Optimal: {optimal_angle}°')
ax1.set_xlabel('Launch Angle (degrees)', fontsize=12)
ax1.set_ylabel('Range (m)', fontsize=12)
ax1.set_title('Range vs Launch Angle', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11)

# Max height vs angle
ax2.plot(angle_df['angle'], angle_df['max_height_m'], 's-', linewidth=2.5,
         markersize=10, color='green')
ax2.set_xlabel('Launch Angle (degrees)', fontsize=12)
ax2.set_ylabel('Maximum Height (m)', fontsize=12)
ax2.set_title('Max Height vs Launch Angle', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('angle_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Analysis plot saved as 'angle_analysis.png'")

## 3. Study Effect of Initial Velocity

Now let's study how initial velocity affects range at the optimal 45° angle.

In [ ]:
# Run parametric study varying velocity
print("Running parametric study: Effect of initial velocity...")
velocities = [10, 15, 20, 25, 30]

results_velocity = fz.fzr(
    "ProjectileMotion.mo",
    {"velocity": velocities, "launch_angle": 45.0},
    "Modelica",
    calculators="localhost",
    results_dir="results_velocity"
)

print("\nResults summary:")
display(results_velocity[['velocity', 'launch_angle', 'status']])

In [ ]:
# Calculate metrics for each velocity
velocity_analysis = []

for idx, row in results_velocity.iterrows():
    if row['status'] == 'done' and 'res' in row:
        velocity = row['velocity']
        res_data = row['res']['ProjectileMotion']
        
        x_vals = np.array(list(res_data['x'].values()))
        y_vals = np.array(list(res_data['y'].values()))
        t_vals = np.array(list(res_data['time'].values()))
        
        # Find landing point
        landing_idx = np.where(y_vals < 0)[0]
        if len(landing_idx) > 0:
            landing_idx = landing_idx[0] - 1
        else:
            landing_idx = len(y_vals) - 1
        
        range_distance = x_vals[landing_idx]
        flight_time = t_vals[landing_idx]
        max_height = np.max(y_vals[y_vals >= 0])
        
        # Theoretical range: R = v²sin(2θ)/g for θ=45°, R = v²/g
        theoretical_range = velocity**2 / 9.81
        
        velocity_analysis.append({
            'velocity': velocity,
            'range_m': range_distance,
            'theoretical_range_m': theoretical_range,
            'error_percent': abs(range_distance - theoretical_range) / theoretical_range * 100,
            'max_height_m': max_height,
            'flight_time_s': flight_time
        })

velocity_df = pd.DataFrame(velocity_analysis)
print("\nVelocity Analysis (θ = 45°):")
print("="*90)
display(velocity_df.round(2))

print("\n📊 Observations:")
print(f"  • Range scales with v² (quadratic relationship)")
print(f"  • Simulation vs Theory error: {velocity_df['error_percent'].mean():.2f}% average")

In [ ]:
# Plot velocity effects
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Trajectories
ax = axes[0, 0]
for idx, row in results_velocity.iterrows():
    if row['status'] == 'done' and 'res' in row:
        velocity = row['velocity']
        res_data = row['res']['ProjectileMotion']
        x_vals = list(res_data['x'].values())
        y_vals = list(res_data['y'].values())
        valid_points = [(x, y) for x, y in zip(x_vals, y_vals) if y >= -0.1]
        if valid_points:
            x_plot, y_plot = zip(*valid_points)
            ax.plot(x_plot, y_plot, marker='o', markersize=3, linewidth=2,
                   label=f'v₀ = {velocity} m/s', alpha=0.8)
ax.set_xlabel('Distance (m)', fontsize=11)
ax.set_ylabel('Height (m)', fontsize=11)
ax.set_title('Trajectories for Different Velocities', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)

# Range vs velocity
ax = axes[0, 1]
ax.plot(velocity_df['velocity'], velocity_df['range_m'], 'o-', linewidth=2.5, 
        markersize=10, color='blue', label='Simulated')
ax.plot(velocity_df['velocity'], velocity_df['theoretical_range_m'], 's--', 
        linewidth=2, markersize=8, color='red', alpha=0.7, label='Theoretical (v²/g)')
ax.set_xlabel('Initial Velocity (m/s)', fontsize=11)
ax.set_ylabel('Range (m)', fontsize=11)
ax.set_title('Range vs Velocity (θ=45°)', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Max height vs velocity
ax = axes[1, 0]
ax.plot(velocity_df['velocity'], velocity_df['max_height_m'], 's-', linewidth=2.5,
        markersize=10, color='green')
ax.set_xlabel('Initial Velocity (m/s)', fontsize=11)
ax.set_ylabel('Maximum Height (m)', fontsize=11)
ax.set_title('Max Height vs Velocity', fontweight='bold')
ax.grid(True, alpha=0.3)

# Flight time vs velocity
ax = axes[1, 1]
ax.plot(velocity_df['velocity'], velocity_df['flight_time_s'], 'd-', linewidth=2.5,
        markersize=10, color='purple')
ax.set_xlabel('Initial Velocity (m/s)', fontsize=11)
ax.set_ylabel('Flight Time (s)', fontsize=11)
ax.set_title('Flight Time vs Velocity', fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('velocity_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Velocity analysis saved as 'velocity_analysis.png'")

## 4. Key Findings

### Effect of Launch Angle:
1. **45° is optimal** for maximum range (in vacuum, no air resistance)
2. **Symmetric relationship**: Complementary angles (30° & 60°, 15° & 75°) give same range but different heights
3. **Height vs Range tradeoff**: Steeper angles achieve greater height but shorter range

### Effect of Initial Velocity:
1. **Quadratic scaling**: Range ∝ v₀² (doubling velocity quadruples range)
2. **Excellent agreement** with theory: R = v²sin(2θ)/g
3. **Linear flight time**: Flight time scales linearly with velocity

### Classical Mechanics Validation:
The simulation results match theoretical predictions, validating our Modelica model!

## Next Steps

Try exploring:
- Add air resistance (drag force)
- Vary gravitational acceleration (other planets)
- Launch from elevated positions
- Moving targets